# KNN Recommender System Application: Missing Birds
## Inputs
Read in our training and testing data inputs

In [15]:
#!/bin/python3
from rich.progress import track

import time
from datetime import datetime

import numpy as np
from common import *

train_set = read_input('../train/train_set.csv')
test_set  = read_input('../test/test_set.csv')

By pre-processing `train_set` and `test_set`, we can reduce the amount of noise we have to deal with.

In [16]:
#> clipping our sets
train_set['data'] = np.clip(train_set['data'], a_min = None, a_max = 10)
test_set['data'] = np.clip(test_set['data'], a_min = None, a_max = 10)

## Outputs
Now we need to create our recommend_set array, which will hold the final output

In [17]:
recommend_set = np.zeros(test_set['data'].shape, dtype = int)

We also need some intermediary arrays to select the 5 birds we are recommending per `test_set` sample (Note: samples are also referred to as columns from now on).

In [18]:
k = 20
ground_truth = 5

knn_graph     = np.zeros((train_set['width'], test_set['width']), dtype = int)
knn_index     = np.zeros((train_set['width'], k), dtype = int)
knn_centroids = np.zeros(train_set['height'])


#todo: we are still working on this script so the code is pasted below

## KNN Algorithm
This is our KNN algorithm along with our distance functions

In [ ]:
def euclidean_distance(x, y):
    return np.sqrt(np.sum(np.square(np.subtract(x, y))))

def cosine_distance(x, y):
    a = np.sum(np.multiply(x, y))
    b = np.multiply( np.sqrt(np.sum( np.square(x))), np.sqrt( np.sum( np.square(y))))
    return 1 - np.divide(a, b)

def jaccard_distance(x, y):
    a = np.sum(np.multiply(x, y))
    b = np.add(np.sum(np.square(x)), np.sum(np.square(y)))
    return np.divide(a, np.subtract(b, a))

def absolute_difference(x, y):
    return np.subtract(x, y)

def KNN(train_set, test_sample, k):
    neighbor_distances = np.zeros(train_set['width'])
    nearest_neighbors  = np.zeros(train_set['width'], dtype = int)

    for column in train_set['columns']:
        neighbor_distances[column] = euclidean_distance(train_set['data'][:,column], test_sample)

    nearest_index = np.argpartition(neighbor_distances, k)[:k]

    # for i in range(len(neighbor_distances)):
        # nearest_neighbors[i] = 1 if neighbor_distances[i] in neighbor_distances[nearest_index] else 0
    for i in nearest_index:
        nearest_neighbors[i] = 1

    # print(f'nearest neighbors: {nearest_neighbors[nearest_index]}')
    # print(f'assert sum is k = {np.sum(nearest_neighbors)}')
    assert np.sum(nearest_neighbors) == k

    return nearest_neighbors

In [ ]:
# exit_value = 10
exit_value = test_set['width'] - 1

print(f'finding k nearest neighbors...') # note: 5 tc's costs 0.7856 seconds, 770 seconds for all tc's
start = time.time()
for column in track(test_set['columns']):
# for column in test_set['columns']:
    knn_graph[:,column] = KNN(train_set, test_set['data'][:,column], k)

    #> find our index of all k 1's in `knn_graph` column
    knn_index = np.argwhere(knn_graph[:,column] == 1)

    #> find our centroids for all `train_set` rows
    for row in train_set['rows']:
        knn_centroids[row] = np.mean(train_set['data'][row, knn_index])

    #> find our index of zero-values in a `test_set` column
    test_sample_zero_index = np.where(test_set['data'][:,column] == 0)[0]

    #> find index of largest values in our centroids AND that are zero in the `test_set` column
    recommend_set_index = test_sample_zero_index[np.argpartition(-knn_centroids[test_sample_zero_index], ground_truth)[:ground_truth]]

    for i in recommend_set_index:
        recommend_set[i, column] = 1

    assert np.sum(recommend_set[:,column]) == ground_truth

    if column == exit_value:
        break

end = time.time()
print(f'timelapse: {time.strftime("%H hours, %M minutes, %S seconds", time.gmtime(end - start))}')

#> write to our .csv file
now = datetime.now()
write_output(f'../out/recommend_set_{now.strftime("%m_%d_%H_%M_%S")}.csv', recommend_set)
